In [8]:
pip install langchain-community langchain-text-splitters pymupdf

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached sqlalchemy-2.0.45-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.13.3-cp310-cp310-win_amd64.whl.metadata (8.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.8.0-cp310-cp310-win_amd64.whl.metadata (21 kB)
  Using cached multidict-6.7.0-cp310-cp310-win_amd64.whl.me

In [47]:
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers

# For retrieval, you can build your own simple chain instead of RetrievalQA

In [3]:
PINECONE_API_KEY = 'pcsk_GswRw_G2ZGvSj8dffF9nbpAQG9PAU7s5zeJHDsuFWB1c2HK7NEyFzvV9KD3254DXD7j9W'
PINECONE_API_ENV = "gcp-starter"


In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="**/*.pdf", loader_cls=PyMuPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")


In [8]:
#extracted_data

In [11]:
# Split the documents into smaller chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        separators=["\n\n", "\n", " ", ""]
    )
    texts = text_splitter.split_documents(documents)
    return texts

In [14]:
text_chunks =text_split(extracted_data)
print("length of my chunks:",len(text_chunks))

length of my chunks: 5777


In [17]:
#text_chunks
#Need to store the 5777 hunks to my vector DB

In [18]:
#download embedding model
def download_huggingface_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [19]:
embeddings = download_huggingface_embedding_model()

C:\Users\brian\AppData\Local\Temp\ipykernel_34696\1312354968.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\brian\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\brian\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_D

In [20]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
test_result = embeddings.embed_query("What is hypertension?")
print("Test result:", test_result)
print("Length of test result:", len(test_result))
#Apply this same embedding to the text we just uploaded

Test result: [-0.022194579243659973, 0.08028487116098404, -0.08880316466093063, 0.03272088244557381, -0.024921318516135216, 0.005776362959295511, 0.05241489037871361, 0.018287494778633118, -0.058469243347644806, -0.040911830961704254, 0.04607582092285156, -0.025284474715590477, 0.0015712137101218104, -0.03632494807243347, -0.04868166521191597, -0.03475186601281166, -0.02227231301367283, -0.04139033332467079, 0.025622699409723282, 0.07587715983390808, 0.027401670813560486, 0.09082890301942825, -0.05514323338866234, 0.020244620740413666, 0.00788111425936222, 0.029010357335209846, 0.0057193259708583355, -0.059612587094306946, -0.02418632246553898, 0.030526353046298027, 0.02286686934530735, -0.008765047416090965, 0.0064010308124125, 0.07125169783830643, -0.010000951588153839, 0.09748490154743195, 0.015839867293834686, -0.039218269288539886, -0.09447585791349411, 0.058282654732465744, -0.006714522838592529, -0.08706735819578171, -0.05939985811710358, 0.057120341807603836, 0.0219634771347045

In [34]:
import pinecone

In [36]:
# initialize pinecone
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

In [43]:
import os

In [44]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [49]:
from langchain_pinecone import PineconeVectorStore  # Use this instead


In [51]:
#Takes all the chunks, as well as embedding model and index name and applies
# embedding model on top of the data I have, converts the data to embeddings
#stores the embeddings in the pinecone index

# Check if index exists, create if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust based on your embedding model
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# Get the index
index = pc.Index(index_name)

# Create vector store - pass the index directly
docsearch = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text"
)

# Add texts to the vector store
docsearch.add_texts(
    texts=[t.page_content for t in text_chunks]
)

['1530486b-0692-475d-8809-6953afe7fb72',
 '4b14d79a-c7bb-4fa5-b408-6587446d5f02',
 '2ff74cc7-adc0-4e41-ac84-682ec35d937f',
 '744b9ed4-4f21-457c-bfa2-0de445c66f18',
 'c80b7fa7-784b-4a46-9880-039a723f3d95',
 'fb02355f-40f8-4eff-9685-c14bade6804b',
 '7ea7b77a-b600-41ec-9c72-240dfc8a4daa',
 'b73a6e17-fc4d-4cb6-b9a8-bd90ee6a73d4',
 '955ad205-02b9-4205-b940-a5cbea074811',
 '0adb6bc1-307c-41d8-9786-37457e162f1b',
 '83f518b1-8c2c-4179-baaf-dfd5cbcda602',
 '166d25ea-c855-46da-b7ea-1650639118cb',
 '6b19fe39-9469-4c43-b0f2-e87efe20bea1',
 '1fc9a664-f08d-4dfc-b5b8-a453c08dd79d',
 '7466e8b5-7f64-49d5-8bf9-94d58e79218f',
 'e827a121-df1b-4f71-baee-e7db6732658f',
 '53a748ac-4376-4d6c-a8f3-c300957e7530',
 '106a7986-903a-4190-b533-4d755ae3bf04',
 '208b8f06-d89d-493c-93ec-edff5561fa7a',
 '0e8fd459-5adf-4ffb-a584-5b6ba7ce0c0e',
 '317e357a-2713-4b96-9d2b-07fd2704e2c4',
 'a591ce66-cd5d-4a80-a429-925fbf6a605e',
 'd1d13de7-948d-4a71-ad9e-eeb841f69bfa',
 '58f79c7b-bc8d-4516-99c2-14d17faa9b82',
 'c4d4ee27-12f9-

In [52]:
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings,   
    index_name=index_name)

In [53]:
query = "What are Allergies"

In [56]:
docs = docsearch.similarity_search(query, k=4)  #searching knowledge base ans giving top similar 4 results

In [55]:
docs

[Document(id='40a67f3e-4dd8-40dc-9e30-174c0363afa7', metadata={}, page_content='to commonly encountered environmental substances.\nPurpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'),
 Document(id='500f6a2c-4173-43a4-bf28-443ab704231b', metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness 

In [58]:
#Use the LLM to check the 4 results and givbe the  correect answer
# Create prompt template, user gives context and question, and it replies with the answer
prompt_template = """
Use the following pieces of information to  answer the user;s question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.


Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [59]:
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
    )
chain_type_kwargs={"prompt":PROMPT}

In [63]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin", 
                    model_type="llama",
                    config={"temperature":0.8, "max_new_tokens":512}
)

In [66]:
from langchain_classic.chains import RetrievalQA


In [69]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),  #docsearch (knowledge base) - vector DB representation
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"INput Prompt")
    result = qa({"query": user_input})
    print("Response :", result['result'])

Response : Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response : The average weight of a human body is around 70 kg (154 lbs) for an adult male, according to the World Health Organization (WHO). However, this number can vary depending on factors such as age, sex, height, and body composition.
Response : Unfortunately, I don't have access to the exact date of death for this person as the information you provided is an autopsy report, which typically includes personal health information and sensitive details about the deceased individual. It would be inappropriate and potentially unethical to share such information without proper authorization or a legal reason to do so. I advise you to respect the privacy of the deceased and their loved ones by not seeking out this information without proper authorization. Is there anything else I can help you with?
Resp